In [1]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# train url : https://drive.google.com/file/d/1VjtFvAkIwCGNmUsK2hquayN0GjuUP1vZ/view?usp=sharing
# test url : https://drive.google.com/file/d/18rNhxsauakGajIE1mjedwnQvWlR4ibXF/view?usp=sharing
download = drive.CreateFile({"id": "1VjtFvAkIwCGNmUsK2hquayN0GjuUP1vZ"}) #이부분 수정하면 됩니당
download.GetContentFile("train.pkl")
download = drive.CreateFile({"id": "18rNhxsauakGajIE1mjedwnQvWlR4ibXF"}) #이부분 수정하면 됩니당
download.GetContentFile("test.pkl")

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split



train = pd.read_pickle("./train.pkl")
test = pd.read_pickle("./test.pkl")
train.shape, test.shape
randomstate =1017

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import x
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score



models = [
    
    # GradientBoostingClassifier(random_state = randomstate),
    
    # RandomForestClassifier(random_state = randomstate),
    
    # LogisticRegression(random_state = randomstate),
    
    # KNeighborsClassifier(3), 
    
    # SVC(probability=True,random_state = randomstate),

    XGBClassifier(random_state = randomstate, objective='multi:softmax'),
    lgb.LGBMClassifier(random_state = randomstate , objective='multiclass') ,
    CatBoostClassifier(random_state = randomstate,loss_function='MultiClass')
]

# names= ['gbc','rfc','lr', 'knn', 'svc' , 'xgb', 'lgbm', 'cat' ]

names= ['xgb', 'lgbm', 'cat' ]


skf = list(StratifiedKFold(n_splits=5,random_state=randomstate, shuffle = True).split(X, y) )

def accu_cv(model,X,y):
    acc = np.sqrt(cross_val_score(model, X, y, cv=skf, scoring = 'accuracy'))
    return acc

for name, model in zip(names, models):
    score = accu_cv(model, train[cols], train['label'] )
    print("{}: {:.6f}".format(name,score.mean())) 
    


In [22]:
#preprocessing  - train/test/split 



avoid_cols = ['label']

cols = [col for col in train.columns if col not in avoid_cols]


X= train[cols]

y= train[['label']]

y = y-3 

X_train, X_val, y_train, y_val = train_test_split(train[cols], 

                                                    train['label'] , 

                                                    test_size=0.3, 
                                                  
                                                    shuffle = True,

                                                    stratify=train['label'], 

                                                    random_state=randomstate)

X_train.head()



skf = list(StratifiedKFold(n_splits=5,random_state=randomstate, shuffle = True).split(X_train, y_train) )





In [ ]:
#Optimization process 

from sklearn.model_selection import RandomizedSearchCV 
from scipy.stats import randint


#1 XGBoost 

#features need to be one hot encoding 




param_distribs = { 'n_estimators': randint(low=1, high=1000), 
                  'max_depth': randint(low=1, high=20),  #default = 6
                  'learning_rate': np.linspace(0.05, 0.1) }  #default = 0.3 

xgb = XGBClassifier(booster= 'gbtree',
                                objective = 'multi:softmax',
                                eval_metric = 'mlogloss',
                                eval_set = [(X_val, y_val)], 
                                verbosity= 2)



                                

rnd_search = RandomizedSearchCV(xgb ,
                                param_distributions=param_distribs,
                                n_iter=50, 
                                cv=skf, 
                                scoring='accuracy', #regression 
                                random_state=randomstate) 




rnd_search.fit(X_train, y_train)

cvresult= rnd_search.cv_results_ 

[07:21:56] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[07:21:56] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[07:21:56] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[07:21:56] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[07:21:56] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[07:21:56] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[07:21:56] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[07:21:56] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pr

In [ ]:
#2 lgbm 



param_distribs = { 'num_iteration': randint(low=1, high=1000), 
                  'max_depth': randint(low=1, high=20),  #default = 6
                  'learning_rate': np.linspace(0.05, 0.1) }  #default = 0.3 


lgb = LGBMClassifier(random_state = randomstate , objective='multiclass')


                                

rnd_search2 = RandomizedSearchCV(lgb ,
                                param_distributions=param_distribs,
                                n_iter=100, 
                                cv=skf, 
                                scoring='accuracy', #regression 
                                random_state=randomstate) 







rnd_search2.fit(X_train, train_y)

cvresult2= rnd_search.cv_results_ 


In [ ]:
#SVM 


svc = SVC(probability=True,random_state = randomstate)



param_distribs = { 'C': randint(low=1, high=10) }  




rnd_search3 = RandomizedSearchCV(svc ,
                                param_distributions=param_distribs,
                                n_iter=10, 
                                cv=skf, 
                                scoring='accuracy', #regression 
                                random_state=randomstate) 




In [25]:
#STACKING 

from mlxtend.classifier import StackingCVClassifier

#MODELS 

clf1 =  XGBClassifier(booster= 'gbtree',
                                objective = 'multi:softmax',
                                eval_metric = 'mlogloss',
                                eval_set = [(X_val, y_val)], 
                                verbosity= 2)


clf2 = LGBMClassifier(random_state = randomstate , objective='multiclass')
clf3 = SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0, 
           decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf', max_iter=-1, probability=True, random_state=1017, shrinking=True, tol=0.001, verbose=2)  


# clf4 = 

lr = LogisticRegression(solver= 'sag')


#sag : multiclass / liblinear ; binary



#STACKING CV 
sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3 ],
                            meta_classifier=lr,
                            verbose= 2,
                            stratify = skf
                            )

sclf.fit(X_train.values,y_train.values)




Fitting 3 classifiers...
Fitting classifier1: xgbclassifier (1/3)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
              eval_set=[(         0         1         2     ...      3069      3070      3071
560  0.568627  0.415686  0.423529  ...  0.298039  0.137255  0.015686
420  0.384314  0.301961  0.168627  ...  0.803922  0.811765  0.835294
159  0.396078  0.270588  0.211765  ...  0.152941  0.078431  0.074510
473  0.313725  0.321569  0.286275  ...  0....
[180 rows x 3072 columns],
                         560    5
420    5
159    3
473    5
183    3
      ..
250    4
455    5
201    4
173    3
396    4
Name: label, Length: 180, dtype: int32)],
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softmax', random_state=0,
              reg_alpha=0, 

StackingCVClassifier(classifiers=[XGBClassifier(base_score=0.5,
                                                booster='gbtree',
                                                colsample_bylevel=1,
                                                colsample_bynode=1,
                                                colsample_bytree=1,
                                                eval_metric='mlogloss',
                                                eval_set=[(         0         1         2     ...      3069      3070      3071
560  0.568627  0.415686  0.423529  ...  0.298039  0.137255  0.015686
420  0.384314  0.301961  0.168627  ...  0.803922  0.811765  0.835294
159  0.396078  0.270588  0.211765  ...  0.152941  0.078431  0.0745...
        86,  89,  91,  94, 111, 113, 116, 123, 125, 134, 137, 139, 148,
       154, 156, 158, 163, 164, 181, 182, 184, 185, 188, 189, 192, 193,
       194, 196, 201, 203, 213, 215, 223, 225, 235, 240, 244, 245, 249,
       253, 256, 262, 263, 271, 283, 292,

In [32]:
from sklearn.metrics import accuracy_score ,f1_score  ,confusion_matrix
from sklearn.metrics import classification_report

pred = sclf.predict(X_val)

accuracy_score(y_val,pred)
f1_score(y_val,pred, average= 'macro')
print(classification_report(y_val, pred))

ValueError: ignored

In [29]:
sclf.predict(X_val)

ValueError: ignored

In [30]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
80,0.764706,0.882353,0.925490,0.764706,0.882353,0.925490,0.733333,0.854902,0.905882,0.749020,0.874510,0.921569,0.780392,0.901961,0.945098,0.784314,0.913725,0.949020,0.792157,0.917647,0.952941,0.792157,0.894118,0.941176,0.807843,0.894118,0.945098,0.811765,0.901961,0.949020,0.823529,0.909804,0.952941,0.831373,0.921569,0.964706,0.843137,0.929412,0.972549,0.847059,...,0.294118,0.329412,0.345098,0.274510,0.325490,0.337255,0.266667,0.325490,0.341176,0.274510,0.337255,0.356863,0.294118,0.349020,0.384314,0.333333,0.372549,0.423529,0.392157,0.419608,0.494118,0.486275,0.474510,0.580392,0.592157,0.545098,0.682353,0.733333,0.647059,0.819608,0.901961,0.749020,0.921569,0.976471,0.909804,0.976471,0.988235,1.000000,0.988235,1.000000
474,0.564706,0.470588,0.411765,0.490196,0.388235,0.337255,0.352941,0.274510,0.215686,0.364706,0.278431,0.231373,0.470588,0.360784,0.321569,0.517647,0.392157,0.333333,0.392157,0.305882,0.266667,0.227451,0.192157,0.156863,0.352941,0.305882,0.239216,0.407843,0.317647,0.298039,0.376471,0.274510,0.266667,0.368627,0.258824,0.250980,0.376471,0.266667,0.266667,0.396078,...,0.121569,0.305882,0.262745,0.250980,0.329412,0.278431,0.258824,0.356863,0.301961,0.266667,0.298039,0.227451,0.192157,0.274510,0.196078,0.152941,0.254902,0.180392,0.149020,0.219608,0.129412,0.113725,0.231373,0.156863,0.125490,0.411765,0.384314,0.329412,0.862745,0.839216,0.792157,0.917647,0.909804,0.870588,0.941176,0.929412,0.894118,0.905882,0.878431,0.854902
525,0.403922,0.156863,0.066667,0.415686,0.156863,0.066667,0.454902,0.172549,0.078431,0.462745,0.184314,0.094118,0.454902,0.196078,0.090196,0.631373,0.509804,0.392157,0.933333,0.905882,0.847059,1.000000,1.000000,0.980392,0.992157,0.941176,0.788235,0.984314,0.909804,0.713725,0.984314,0.972549,0.780392,0.980392,0.968627,0.760784,0.972549,0.945098,0.678431,0.972549,...,0.282353,0.513725,0.403922,0.274510,0.380392,0.294118,0.203922,0.254902,0.180392,0.121569,0.137255,0.086275,0.050980,0.066667,0.031373,0.011765,0.043137,0.019608,0.000000,0.050980,0.023529,0.007843,0.066667,0.035294,0.015686,0.086275,0.043137,0.019608,0.156863,0.090196,0.066667,0.219608,0.129412,0.094118,0.125490,0.058824,0.035294,0.105882,0.058824,0.035294
131,0.764706,0.650980,0.423529,0.815686,0.701961,0.537255,0.870588,0.772549,0.717647,0.847059,0.807843,0.819608,0.235294,0.207843,0.223529,0.239216,0.200000,0.203922,0.345098,0.294118,0.298039,0.368627,0.294118,0.258824,0.427451,0.341176,0.278431,0.462745,0.380392,0.329412,0.556863,0.447059,0.384314,0.647059,0.513725,0.423529,0.690196,0.564706,0.482353,0.713725,...,0.721569,0.741176,0.701961,0.705882,0.737255,0.694118,0.713725,0.709804,0.666667,0.698039,0.552941,0.478431,0.450980,0.474510,0.364706,0.270588,0.466667,0.360784,0.227451,0.486275,0.396078,0.274510,0.482353,0.407843,0.282353,0.572549,0.474510,0.329412,0.643137,0.525490,0.352941,0.596078,0.458824,0.274510,0.568627,0.427451,0.243137,0.474510,0.325490,0.160784
291,0.101961,0.101961,0.082353,0.109804,0.105882,0.070588,0.176471,0.172549,0.137255,0.403922,0.415686,0.423529,0.549020,0.568627,0.584314,0.600000,0.615686,0.631373,0.603922,0.623529,0.635294,0.596078,0.607843,0.623529,0.607843,0.611765,0.627451,0.372549,0.376471,0.380392,0.235294,0.239216,0.243137,0.603922,0.603922,0.627451,0.588235,0.592157,0.611765,0.592157,...,0.592157,0.596078,0.607843,0.635294,0.615686,0.627451,0.650980,0.607843,0.611765,0.631373,0.592157,0.596078,0.615686,0.603922,0.607843,0.627451,0.321569,0.321569,0.333333,0.066667,0.066667,0.058824,0.309804,0.313725,0.294118,0.305882,0.305882,0.282353,0.313725,0.317647,0.305882,0.274510,0.278431,0.274510,0.243137,0.243137,0.243137,0.184314,0.192157,0.188235
...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
## Import Package
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix,  
from sklearn.model_selection import cross_val_score


models = [LogisticRegression( multi_class='multinomial' ), SVC(), 
          RandomForestClassifier(), GradientBoostingClassifier(),
          XGBClassifier(objective = 'multi:softmax'), CatBoostClassifier(), LGBMClassifier(objective = 'multiclass')]
  
names = ['LogisticRegression', 'SVM', 
         'RandomForest', 'GB',
         'XBG', 'CAT', 'LGBM']

## 하나의 피처 기준으로 고르게 K-Fold 진행
skf = list(StratifiedKFold(n_splits=5,random_state=1017).split(train, train['label'])) 

# define cross validation strategy
def accu_cv(model,X,y):
    acc = np.sqrt(cross_val_score(model, X, y, cv=skf, scoring = 'accuracy'))
    return acc

for name, model in zip(names, models):
    score = accu_cv(model, X, y )
    print()
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

avoid_cols = ['label']
cols = [col for col in X_train.columns if col not in avoid_cols] #store_id, 3,6,12month, month

print("check X_train shape ", X_train[cols].shape)

model = lgb.LGBMClassifier(task = 'train',
                           boosting_type='gbdt',  
                           objective='multiclass',
                           save_binary =True,
                           metric='multi_logloss',
                           random_state=314,
                          )
                                                   
model.fit(X_train[cols], y_train, 
          verbose = False, early_stopping_rounds = 100, eval_metric = 'multi_logloss', 
          eval_set = [(X_train[cols], y_train), (X_val[cols], y_val)], 
          eval_names = ['train', 'valid']
          )

pred = model.predict(X_val[cols])
score = accuracy_score(y_val, pred)

print("SCORE : ", score)

In [31]:
X_val

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
560,0.568627,0.415686,0.423529,0.556863,0.403922,0.415686,0.552941,0.403922,0.423529,0.568627,0.411765,0.443137,0.654902,0.509804,0.537255,0.694118,0.556863,0.576471,0.654902,0.521569,0.552941,0.552941,0.407843,0.431373,0.549020,0.396078,0.423529,0.552941,0.407843,0.419608,0.537255,0.380392,0.384314,0.658824,0.513725,0.521569,0.909804,0.878431,0.878431,0.960784,...,0.341176,0.250980,0.145098,0.113725,0.133333,0.027451,0.000000,0.137255,0.027451,0.000000,0.180392,0.062745,0.000000,0.184314,0.070588,0.007843,0.200000,0.086275,0.007843,0.278431,0.129412,0.027451,0.282353,0.137255,0.027451,0.282353,0.133333,0.027451,0.294118,0.133333,0.023529,0.305882,0.141176,0.019608,0.278431,0.121569,0.015686,0.298039,0.137255,0.015686
420,0.384314,0.301961,0.168627,0.403922,0.321569,0.180392,0.439216,0.352941,0.200000,0.450980,0.368627,0.219608,0.454902,0.372549,0.219608,0.592157,0.541176,0.431373,0.839216,0.870588,0.890196,0.258824,0.243137,0.239216,0.066667,0.035294,0.019608,0.133333,0.098039,0.086275,0.176471,0.125490,0.117647,0.082353,0.047059,0.031373,0.290196,0.215686,0.200000,0.576471,...,0.360784,0.129412,0.113725,0.105882,0.200000,0.192157,0.184314,0.552941,0.560784,0.568627,0.800000,0.823529,0.854902,0.780392,0.803922,0.827451,0.811765,0.831373,0.850980,0.835294,0.854902,0.882353,0.796078,0.807843,0.839216,0.772549,0.784314,0.803922,0.811765,0.823529,0.839216,0.831373,0.847059,0.870588,0.796078,0.803922,0.823529,0.803922,0.811765,0.835294
159,0.396078,0.270588,0.211765,0.505882,0.411765,0.341176,0.717647,0.690196,0.576471,0.745098,0.721569,0.615686,0.764706,0.737255,0.643137,0.690196,0.647059,0.560784,0.627451,0.580392,0.478431,0.549020,0.490196,0.364706,0.631373,0.564706,0.419608,0.643137,0.580392,0.443137,0.580392,0.470588,0.325490,0.529412,0.458824,0.321569,0.752941,0.752941,0.643137,0.788235,...,0.070588,0.313725,0.156863,0.125490,0.325490,0.172549,0.133333,0.313725,0.160784,0.129412,0.333333,0.172549,0.141176,0.341176,0.172549,0.133333,0.341176,0.168627,0.129412,0.325490,0.160784,0.129412,0.298039,0.145098,0.117647,0.278431,0.141176,0.117647,0.266667,0.129412,0.105882,0.219608,0.105882,0.098039,0.176471,0.090196,0.090196,0.152941,0.078431,0.074510
473,0.313725,0.321569,0.286275,0.219608,0.215686,0.207843,0.309804,0.270588,0.290196,0.329412,0.274510,0.298039,0.352941,0.294118,0.325490,0.443137,0.431373,0.450980,0.454902,0.466667,0.450980,0.423529,0.435294,0.400000,0.407843,0.419608,0.380392,0.435294,0.439216,0.403922,0.470588,0.474510,0.454902,0.427451,0.427451,0.403922,0.294118,0.301961,0.282353,0.243137,...,0.074510,0.152941,0.145098,0.105882,0.188235,0.172549,0.137255,0.211765,0.203922,0.164706,0.274510,0.282353,0.250980,0.423529,0.454902,0.423529,0.388235,0.400000,0.368627,0.321569,0.309804,0.266667,0.309804,0.301961,0.258824,0.301961,0.298039,0.254902,0.294118,0.286275,0.254902,0.290196,0.278431,0.250980,0.290196,0.270588,0.243137,0.278431,0.262745,0.243137
183,0.643137,0.619608,0.631373,0.647059,0.627451,0.643137,0.662745,0.643137,0.666667,0.666667,0.650980,0.674510,0.701961,0.694118,0.733333,0.847059,0.839216,0.890196,0.870588,0.862745,0.917647,0.878431,0.874510,0.937255,0.898039,0.894118,0.960784,0.878431,0.878431,0.941176,0.862745,0.862745,0.913725,0.752941,0.749020,0.768627,0.705882,0.694118,0.717647,0.737255,...,0.384314,0.584314,0.403922,0.407843,0.513725,0.352941,0.337255,0.462745,0.290196,0.270588,0.392157,0.313725,0.290196,0.415686,0.368627,0.356863,0.588235,0.552941,0.584314,0.466667,0.439216,0.474510,0.192157,0.145098,0.137255,0.105882,0.066667,0.062745,0.066667,0.031373,0.035294,0.086275,0.043137,0.043137,0.133333,0.054902,0.062745,0.164706,0.054902,0.066667
...,...,...,...,...,...,...,...,...,...,...,.

In [35]:
#STACKING 

from mlxtend.classifier import StackingCVClassifier

#MODELS 

# clf1 =  XGBClassifier(booster= 'gbtree',
#                                 objective = 'multi:softmax',
#                                 eval_metric = 'mlogloss',
#                                 eval_set = [(X_val, y_val)], 
#                                 verbosity= 2)


clf2 = LGBMClassifier(random_state = randomstate , objective='multiclass')
clf3 = SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0, 
           decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf', max_iter=-1, probability=True, random_state=1017, shrinking=True, tol=0.001, verbose=2)  


lr = LogisticRegression(solver= 'sag')



#STACKING CV 
sclf = StackingCVClassifier(classifiers=[ clf2, clf3 ],
                            meta_classifier=lr,
                            verbose= 2,
                            stratify = skf)

sclf.fit(X.values,y.values)




Fitting 2 classifiers...
Fitting classifier1: lgbmclassifier (1/2)
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31,
               objective='multiclass', random_state=1017, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)
Training and fitting fold 1 of 2...


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training and fitting fold 2 of 2...


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting classifier2: svc (2/2)
SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=1017, shrinking=True, tol=0.001,
    verbose=2)
Training and fitting fold 1 of 2...
[LibSVM]

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training and fitting fold 2 of 2...
[LibSVM]

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: ignored

In [ ]:

def predict(test):
  final_pred = sclf.predict(test)
  np.save("./submissin_file.npy", final_pred)

  return final_pred

In [26]:

# VotingClassifier
from sklearn.ensemble import VotingClassifier


clf1 =  XGBClassifier(booster= 'gbtree',
                                objective = 'multi:softmax',
                                eval_metric = 'mlogloss',
                                eval_set = [(X_val, y_val)], 
                                verbosity= 2)


clf2 = LGBMClassifier(random_state = randomstate , objective='multiclass')
clf3 = SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0, 
           decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf', max_iter=-1, probability=True, random_state=1017, shrinking=True, tol=0.001, verbose=2)  

lr = LogisticRegression(solver= 'sag')

models = [('xgb',clf1),
          ('lgb',clf2),
          ('svc',clf3),
          ('lr',lr)]

hard_vote  = VotingClassifier(models, voting='hard')

In [12]:
X_train.shape

(420, 3072)

In [13]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.optimizers import Adam


# def dnn_model():
#     model = Sequential()

#     model.add(Dense(64,input_shape=X_train.shape,activation='relu'))
#     model.add(Dense(64,activation='relu'))
#     model.add(Dense(3,activation='softmax')) 

# #sofmax: multi-class(last layer= # class )  , loss = 'categorical-crossentropy'
# #binary: loss= binary_crossentropy
# #regression : model.add(Dense(1, activation='sigmoid')) . last layer = 1 , loss = 'mse'

#     model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])

#     model.summary()
    
#     return model 


    
# def train_dnn():
#     hist = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100)

#     return hist 
    
    
    
# # plt.figure(figsize=(12,8))
# # plt.plot(hist.history['loss'])
# # plt.plot(hist.history['val_loss'])
# # plt.plot(hist.history['acc'])
# # plt.plot(hist.history['val_acc'])
# # plt.legend(['loss','val_loss', 'acc','val_acc'])
# # plt.grid()
# # plt.show()

# # model.evaluate(X_test, y_test)

In [9]:
y_train = y_train-1
y_val = y_val-1

In [14]:
model = dnn_model()
train_dnn()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 420, 64)           196672    
_________________________________________________________________
dense_10 (Dense)             (None, 420, 64)           4160      
_________________________________________________________________
dense_11 (Dense)             (None, 420, 3)            195       
Total params: 201,027
Trainable params: 201,027
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100


ValueError: ignored